<a href="https://colab.research.google.com/github/Sopralapanca/CommonLitChallenge/blob/Sarde/simple_two_layer_bidirectional_lstm_with_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple two-layer bidirectional LSTM with Pytorch

In [169]:
!pip install transformers
!pip install keras

In [170]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import time
import copy
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
from torch.optim.lr_scheduler import ReduceLROnPlateau

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from spacy.lang.en import English
import string
import spacy
from torch.utils.data import Dataset, DataLoader
from transformers import BertPreTrainedModel, BertModel
from transformers import AutoConfig, AutoTokenizer
import torch

from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification,AutoTokenizer

from torch import nn

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from transformers import DebertaModel
from transformers import DebertaPreTrainedModel
from tqdm import tqdm, trange
import os
import gc
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn.functional as F
import torchtext
import nltk
import time
from datetime import timedelta
import numpy as np
from sklearn import metrics
import os
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence
from torch import Tensor

#Get dataset




In [171]:
!git clone "https://github.com/Sopralapanca/CommonLitChallenge.git"

fatal: destination path 'CommonLitChallenge' already exists and is not an empty directory.


In [172]:
import string
import re
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import seaborn as sns
import matplotlib.pyplot as plt
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [173]:
!pip install pyspellchecker

In [174]:
!pip install swifter

In [175]:
import swifter

# Load data

### Load the prompt csv

In [176]:
df_train_prompt = pd.read_csv('/content/CommonLitChallenge/data/prompts_train.csv')

The data is composed of 4 columns and 4 entries. We can see all 4 dimensions of our dataset by using the following code:

In [177]:
df_train_prompt.head()

,prompt_id,prompt_question,prompt_title,prompt_text
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \nAs the sequel to what has already...
1,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
2,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \nThe Third Wave experiment took pl...
3,ebad26,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."


### Load the summaries csv

In [178]:
df_train_summaries = pd.read_csv('/content/CommonLitChallenge/data/summaries_train.csv')

In [179]:
df_train_summaries.head()

,student_id,prompt_id,text,content,wording
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757


# Data preprocessing methods

In [180]:
# normalize the data between 0 and 1 taking into consideration the prompt title

from sklearn.preprocessing import MinMaxScaler

def normalize_col(df, col):
    # Create a Min-Max Scaler
    scaler = MinMaxScaler()
    df[col] = df.groupby('prompt_id')[col].transform(lambda x: scaler.fit_transform(x.values.reshape(-1, 1)).flatten())

    return df

In [181]:
# get list of all stopwords in english
import nltk
nltk.download('stopwords')
stopword_list = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [182]:
def preprocessText(text, removal=True):
    """
    preprocess text in order to make it lower, without punctuation and stopowrods
    param text: String to be preprocessed
    param removal: boolean, if True stopwords will be removed
    return text: String, preprocessed text
    """

    # replace newline with space
    text = text.replace("\n", " ")
    text = text.replace('\r', ' ')

    text = str(text)

    # Normalize spaces around punctuation marks
    text = re.sub(r"[^A-Za-z0-9']", r' ', text)
    text = re.sub(r'\s+', ' ', text)

    # Replace curly apostrophe with straight single quote
    text = text.replace('’', "'")

    # Normalize spaces around punctuation marks
    text = text.strip()

    # lower case
    text = text.lower()

    if removal:

        # split text
        words = text.split()

        # stop word removal
        words = [w for w in words if not w in stopword_list]

        # stemming
        # words = [stemmer.stem(w) for w in words]

        # lemmatization
        #words = [lemmatizer.lemmatize(w) for w in words]

        text = ' '.join(words)

    return text

In [183]:
def add_row(df1, df2, preprocess=False):
    #append row on the head of the dataframe

    row = df2.unique().tolist()[0]

    if preprocess:
        # apply text preprocessing on the row, removing punctuation, stopwrods...
        row = preprocessText(row)

    combined_data = pd.concat([pd.Series([row]),df1.loc[:]]).reset_index(drop=True) #append row on the head of the dataframe

    return combined_data

In [184]:
# Count the stop words in the text.
def count_stopwords(text: str) -> int:
    words = text.split()
    stopwords_count = sum(1 for word in words if word.lower() in stopword_list)
    return stopwords_count

# Count the punctuations in the text.
# punctuation_set -> !"#$%&'()*+, -./:;<=>?@[\]^_`{|}~
def count_punctuation(text: str) -> int:
    punctuation_set = set(string.punctuation)
    punctuation_count = sum(1 for char in text if char in punctuation_set)
    return punctuation_count

def different_word_counter(row):
    row = preprocessText(row, removal=False)
    words_list = []
    for w in row.split(' '):
        if w not in words_list:
            words_list.append(w)
    return len(words_list)

def count_words(text):
    return len(text.split(' '))


In [185]:
# ref https://stats.stackexchange.com/questions/570698/should-i-remove-stopwords-before-generating-n-grams
# first find ngrams, then remove the one with stopowrds

from nltk.util import ngrams
from nltk.tokenize import word_tokenize

def count_ngrams(text, n):
    text = preprocessText(text, False)

    keep_ngrams = []
    tokens = word_tokenize(text)
    try:
        n_grams = list(ngrams(tokens, n))
        all_ngrams =  [' '.join(gram) for gram in n_grams]

        # remove ngrams that starts or end with stopwords
        for ngram in all_ngrams:
            if not ngram[0] in stopword_list or not ngram[-1] in stopword_list:
                keep_ngrams.append(ngram)
    except:
        pass

    return keep_ngrams

def count_cooccurring_ngrams(text_ngrams, prompt_text_ngrams):
    # Find the common n-grams between the two columns
    common_ngrams = set(text_ngrams) & set(prompt_text_ngrams)
    return len(common_ngrams)

In [186]:
from sklearn.feature_extraction.text import TfidfVectorizer
import sympy

# Group by 'prompt_id' and compute TF-IDF separately for each class
def compute_TFIDF(df):
    tfidf_vectorizers = {}
    karp_tfidf_scores = {}

    for class_id, group in df.groupby('prompt_id'):
        text_data = group['text_nostop']

        prompt_question_data = group['prompt_question']
        prompt_title_data = group['prompt_title']
        prompt_text_data = group['prompt_text_nostop']

        # Concatenate the preprocessed data for TF-IDF calculation
        combined_data = add_row(text_data, prompt_question_data, True)
        combined_data = add_row(combined_data, prompt_title_data, True)
        combined_data = add_row(combined_data, prompt_text_data)


        # Compute TF-IDF
        tfidf_vectorizer = TfidfVectorizer()
        tfidf_matrix = tfidf_vectorizer.fit_transform(combined_data)
        tfidf_vectorizers[class_id] = {'vectorizer': tfidf_vectorizer, 'matrix': tfidf_matrix}

        tfidf_matrix = tfidf_matrix[3:] #remove first 3 rows f the matrix since they belongs to prompt_text, prompt_question, prompt_title

        modulus = sympy.randprime(tfidf_matrix.shape[1]-100, tfidf_matrix.shape[1])

        # Iterate through documents and calculate TF-IDF scores
        for index, row in group.iterrows():
            doc_tfidf = tfidf_matrix[index - group.index[0]].toarray()[0]

            doc_tfidf = doc_tfidf[doc_tfidf>0]

            gamma = 1e-2
            single_tfidf_score = sum([t**(gamma*i) for i, t in enumerate(doc_tfidf)]) % modulus

            karp_tfidf_scores[index] = single_tfidf_score

    # Add the calculated average TF-IDF scores as a new column to the DataFrame
    df['karp_tfidf_scores'] = [karp_tfidf_scores[index] for index in df.index]

    return df

# Preprocessing prompt dataframe before merging

In [187]:
def prompt_feature_engineer(dataframe: pd.DataFrame, feature: str = 'prompt_text') -> pd.DataFrame:
    dataframe[f'{feature}_word_cnt'] = dataframe[feature].swifter.apply(count_words)
    dataframe[f'{feature}_length'] = dataframe[feature].swifter.apply(len)
    dataframe[f'{feature}_stopword_cnt'] = dataframe[feature].swifter.apply(count_stopwords)
    dataframe[f'{feature}_punct_cnt'] = dataframe[feature].swifter.apply(count_punctuation)

    # vedere se prima effetturare preprocess del testo
    dataframe[f'{feature}_different_word_cnt'] = dataframe[feature].swifter.apply(different_word_counter)

    dataframe[f'{feature}_nostop'] = dataframe[feature].swifter.apply(preprocessText)

    # ngrams finding
    for n in range(2,5):
        col = f"{feature}_{n}grams"
        dataframe[col] = dataframe.swifter.apply(lambda row: count_ngrams(row[feature], n), axis=1)


    return dataframe

In [188]:
import nltk
nltk.download('punkt')
df_train_prompt = prompt_feature_engineer(df_train_prompt)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Pandas Apply:   0%|          | 0/4 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/4 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/4 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/4 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/4 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/4 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/4 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/4 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/4 [00:00<?, ?it/s]

# Merging dataframe and extract features

In [189]:
df_train = df_train_summaries.merge(df_train_prompt, on='prompt_id')

In [190]:
import itertools

word_list = []
for prompt, query, title in zip(df_train_prompt.prompt_text.tolist(), df_train_prompt.prompt_question.tolist(), df_train_prompt.prompt_title.tolist()):
    word_list.append(prompt.replace('\n', ' ').split())
    word_list.append(query.replace('\n', ' ').split())
    word_list.append(title.replace('\n', ' ').split())

token_list = list(itertools.chain(*word_list)) #qui mi sembra che ci siano dei token ripetuti, va bene?

**Quick basic dataset exploration**

In [191]:
df_train.describe()

,content,wording,prompt_text_word_cnt,prompt_text_length,prompt_text_stopword_cnt,prompt_text_punct_cnt,prompt_text_different_word_cnt
count,7165.000000,7165.000000,7165.000000,7165.000000,7165.000000,7165.000000,7165.000000
mean,-0.014853,-0.063072,688.625960,3875.733845,339.530216,86.582275,297.882763
std,1.043569,1.036048,173.705465,784.383272,108.475611,18.936104,43.306856
min,-1.729859,-1.962614,550.000000,3329.000000,237.000000,55.000000,262.000000
25%,-0.799545,-0.872720,550.000000,3329.000000,237.000000,73.000000,262.000000
50%,-0.093814,-0.081769,604.000000,3360.000000,305.000000,96.000000,285.000000
75%,0.499660,0.503833,966.000000,5132.000000,509.000000,108.000000,366.000000
max,3.900326,4.310693,966.000000,5132.000000,509.000000,108.000000,366.000000


In [192]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7165 entries, 0 to 7164
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   student_id                      7165 non-null   object 
 1   prompt_id                       7165 non-null   object 
 2   text                            7165 non-null   object 
 3   content                         7165 non-null   float64
 4   wording                         7165 non-null   float64
 5   prompt_question                 7165 non-null   object 
 6   prompt_title                    7165 non-null   object 
 7   prompt_text                     7165 non-null   object 
 8   prompt_text_word_cnt            7165 non-null   int64  
 9   prompt_text_length              7165 non-null   int64  
 10  prompt_text_stopword_cnt        7165 non-null   int64  
 11  prompt_text_punct_cnt           7165 non-null   int64  
 12  prompt_text_different_word_cnt  71

**Preprocess the data**

In [193]:
from spellchecker import SpellChecker

spell = SpellChecker()

spell.word_frequency.load_words(token_list)

def misspelled(text):
    words = text.split()
    misspelled = spell.unknown(words)
    return len(misspelled)

In [194]:

# This function applies all the above preprocessing functions on a text feature.
def feature_engineer(dataframe: pd.DataFrame, feature: str = 'text') -> pd.DataFrame:
    dataframe[f'{feature}_word_cnt'] = dataframe[feature].swifter.apply(count_words)
    dataframe[f'{feature}_length'] = dataframe[feature].swifter.apply(len)
    dataframe[f'{feature}_stopword_cnt'] = dataframe[feature].swifter.apply(count_stopwords)
    dataframe[f'{feature}_punct_cnt'] = dataframe[feature].swifter.apply(count_punctuation)

    # vedere se prima effetturare preprocess del testo
    dataframe[f'{feature}_different_word_cnt'] = dataframe[feature].swifter.apply(different_word_counter)



    # misspelled counter
    dataframe[f'{feature}_misspelled_cnt'] = dataframe[feature].swifter.apply(lambda x: misspelled(x))

    # ratios
    dataframe[f'{feature}_word_ratio'] = dataframe[f'{feature}_word_cnt'] / dataframe['prompt_text_word_cnt']
    dataframe.drop(['prompt_text_word_cnt'], axis=1, inplace=True)

    dataframe[f'{feature}_length_ratio'] = dataframe[f'{feature}_length'] / dataframe['prompt_text_length']
    dataframe.drop(['prompt_text_length'], axis=1, inplace=True)

    dataframe[f'{feature}_stopword_ratio'] = dataframe[f'{feature}_stopword_cnt'] / dataframe['prompt_text_stopword_cnt']
    dataframe.drop(['prompt_text_stopword_cnt'], axis=1, inplace=True)

    dataframe[f'{feature}_punct_ratio'] = dataframe[f'{feature}_punct_cnt'] / dataframe['prompt_text_punct_cnt']
    dataframe.drop(['prompt_text_punct_cnt'], axis=1, inplace=True)


    # vedere se prima effetturare preprocess del testo
    dataframe[f'{feature}_different_word_cnt'] = dataframe[feature].swifter.apply(lambda row: different_word_counter(row))

    dataframe[f'{feature}_different_word_ratio'] = dataframe[f'{feature}_different_word_cnt'] / dataframe['prompt_text_different_word_cnt']
    dataframe.drop(['prompt_text_different_word_cnt'], axis=1, inplace=True)


    dataframe[f'{feature}_nostop'] = dataframe[feature].swifter.apply(preprocessText)
    dataframe = compute_TFIDF(dataframe)

    dataframe.drop(['text_nostop', 'prompt_text_nostop'], axis=1, inplace=True)

    normalize_cols = [f'{feature}_word_cnt',f'{feature}_length',f'{feature}_stopword_cnt',
                      f'{feature}_punct_cnt', f'{feature}_misspelled_cnt',
                     f'{feature}_different_word_cnt', f'{feature}_different_word_ratio',
                     f'{feature}_word_ratio',f'{feature}_length_ratio', f'{feature}_stopword_ratio',
                     f'{feature}_punct_ratio', "karp_tfidf_scores"]

    # ngrams finding
    for n in range(2,5):
        col = f"{feature}_{n}grams"
        dataframe[col] = dataframe.swifter.apply(lambda row: count_ngrams(row[feature], n), axis=1)

        # ngrams coocurrence count
        dataframe[f"{n}grams_cnt"] = dataframe.swifter.apply(lambda row: count_cooccurring_ngrams(row[f"text_{n}grams"], row[f"prompt_text_{n}grams"]), axis=1)

        dataframe.drop([f"text_{n}grams", f"prompt_text_{n}grams"], axis=1, inplace=True)

        normalize_cols.append(f"{n}grams_cnt")


    dataframe = dataframe.replace([np.inf, -np.inf], np.nan).fillna(0)

    for col in normalize_cols:
        dataframe = normalize_col(dataframe, col)


    return dataframe

In [195]:
preprocessed_df = feature_engineer(df_train)

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

...

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

In [196]:
preprocessed_df.head()

,student_id,prompt_id,text,content,wording,prompt_question,prompt_title,prompt_text,text_word_cnt,text_length,...,text_misspelled_cnt,text_word_ratio,text_length_ratio,text_stopword_ratio,text_punct_ratio,text_different_word_ratio,karp_tfidf_scores,2grams_cnt,3grams_cnt,4grams_cnt
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538,Summarize how the Third Wave developed over su...,The Third Wave,Background \nThe Third Wave experiment took pl...,0.099174,0.104635,...,0.081081,0.099174,0.104635,0.087719,0.050847,0.169492,0.529881,0.034188,0.008333,0.000000
1,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757,Summarize how the Third Wave developed over su...,The Third Wave,Background \nThe Third Wave experiment took pl...,0.498623,0.508031,...,0.729730,0.498623,0.508031,0.479532,0.508475,0.610169,0.953371,0.128205,0.033333,0.009259
2,0095993991fe,814d6b,The third wave only started as an experiment w...,0.205683,0.380538,Summarize how the Third Wave developed over su...,The Third Wave,Background \nThe Third Wave experiment took pl...,0.096419,0.104176,...,0.081081,0.096419,0.104176,0.099415,0.152542,0.152542,0.465352,0.059829,0.058333,0.018519
3,00c20c6ddd23,814d6b,The experimen was orginally about how even whe...,0.567975,0.969062,Summarize how the Third Wave developed over su...,The Third Wave,Background \nThe Third Wave experiment took pl...,0.140496,0.152822,...,0.297297,0.140496,0.152822,0.146199,0.169492,0.192090,0.560326,0.102564,0.041667,0.027778
4,00d40ad10dc9,814d6b,The third wave developed so quickly due to the...,-0.910596,-0.081769,Summarize how the Third Wave developed over su...,The Third Wave,Background \nThe Third Wave experiment took pl...,0.005510,0.012391,...,0.054054,0.005510,0.012391,0.011696,0.033898,0.028249,0.160165,0.034188,0.008333,0.000000


In [197]:
preprocessed_df.describe()

,content,wording,text_word_cnt,text_length,text_stopword_cnt,text_punct_cnt,text_different_word_cnt,text_misspelled_cnt,text_word_ratio,text_length_ratio,text_stopword_ratio,text_punct_ratio,text_different_word_ratio,karp_tfidf_scores,2grams_cnt,3grams_cnt,4grams_cnt
count,7165.000000,7165.000000,7165.000000,7165.000000,7165.000000,7165.000000,7165.000000,7165.000000,7165.000000,7165.000000,7165.000000,7165.000000,7165.000000,7165.000000,7165.000000,7165.000000,7165.000000
mean,-0.014853,-0.063072,0.100120,0.098436,0.118987,0.095105,0.146297,0.119816,0.100120,0.098436,0.118987,0.095105,0.146297,0.455192,0.093512,0.065532,0.054993
std,1.043569,1.036048,0.105828,0.102808,0.108247,0.093756,0.118409,0.100842,0.105828,0.102808,0.108247,0.093756,0.118409,0.229856,0.101519,0.095399,0.093689
min,-1.729859,-1.962614,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-0.799545,-0.872720,0.030547,0.030827,0.047826,0.034483,0.063107,0.051282,0.030547,0.030827,0.047826,0.034483,0.063107,0.274284,0.028926,0.007843,0.000000
50%,-0.093814,-0.081769,0.065708,0.064286,0.083045,0.067901,0.112360,0.093023,0.065708,0.064286,0.083045,0.067901,0.112360,0.425575,0.060345,0.027451,0.013514
75%,0.499660,0.503833,0.134185,0.130786,0.156522,0.118644,0.194175,0.153846,0.134185,0.130786,0.156522,0.118644,0.194175,0.620035,0.119658,0.086957,0.073171
max,3.900326,4.310693,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [198]:
preprocessed_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7165 entries, 0 to 7164
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   student_id                 7165 non-null   object 
 1   prompt_id                  7165 non-null   object 
 2   text                       7165 non-null   object 
 3   content                    7165 non-null   float64
 4   wording                    7165 non-null   float64
 5   prompt_question            7165 non-null   object 
 6   prompt_title               7165 non-null   object 
 7   prompt_text                7165 non-null   object 
 8   text_word_cnt              7165 non-null   float64
 9   text_length                7165 non-null   float64
 10  text_stopword_cnt          7165 non-null   float64
 11  text_punct_cnt             7165 non-null   float64
 12  text_different_word_cnt    7165 non-null   float64
 13  text_misspelled_cnt        7165 non-null   float

**Extract feature columns**

In [199]:
FEATURE_COLUMNS = preprocessed_df.drop(columns = ['student_id', 'prompt_id', 'text', 'prompt_question',
                                           'prompt_title', 'prompt_text', 'content', 'wording'], axis = 1).columns.to_list()

In [200]:
FEATURE_COLUMNS

['text_word_cnt',
 'text_length',
 'text_stopword_cnt',
 'text_punct_cnt',
 'text_different_word_cnt',
 'text_misspelled_cnt',
 'text_word_ratio',
 'text_length_ratio',
 'text_stopword_ratio',
 'text_punct_ratio',
 'text_different_word_ratio',
 'karp_tfidf_scores',
 '2grams_cnt',
 '3grams_cnt',
 '4grams_cnt']

# Split and oversample dataset

In [201]:

from sklearn.model_selection import train_test_split
training, test_df = train_test_split(preprocessed_df, test_size=0.2, random_state=42, stratify=preprocessed_df["prompt_id"])
train_df, valid_df= train_test_split(training, test_size=0.2, random_state=42, stratify=training["prompt_id"])

In [202]:
train_df["prompt_id"].value_counts()

39c16e    1317
3b9047    1285
ebad26    1277
814d6b     706
Name: prompt_id, dtype: int64

In [203]:
def oversample_df(df):
    """
    :param df: Dataframe to be overampled based on prompt_id
    :return: Dataframe oversampled
    """
    classes = df["prompt_id"].value_counts().to_dict()
    most = max(classes.values())
    classes_list = []
    for key in classes:
        classes_list.append(df[df["prompt_id"] == key])
    classes_sample = []
    for i in range(1, len(classes_list)):
        classes_sample.append(classes_list[i].sample(most, replace=True))
    df_maybe = pd.concat(classes_sample)
    final_df = pd.concat([df_maybe, classes_list[0]], axis=0)

    return final_df

In [204]:
train_df = oversample_df(train_df)

In [205]:
train_df["prompt_id"].value_counts()

3b9047    1317
ebad26    1317
814d6b    1317
39c16e    1317
Name: prompt_id, dtype: int64

## Define data loaders

In [206]:
class CommonLitDataset(Dataset):
    #TODO: aggiungere max len specifica per tutti e 3 i campi
    def __init__(self, data, maxlen, tokenizer, input_cols, feature_cols, target_cols, split=True, padding=True):
        #Store the contents of the file in a pandas dataframe
        self.df = data.reset_index()
        #Initialize the tokenizer for the desired transformer model
        self.tokenizer = tokenizer
        #Maximum length of the tokens list to keep all the sequences of fixed size
        self.maxlen = maxlen
        #list of input columns
        self.input_cols = input_cols
        #list of target columns
        self.target_cols = target_cols
        #list of feature columns
        self.feature_cols = feature_cols
        self.prompt_text=[]
        self.prompt_question=[]
        self.summary=[]
        self.split=split
        self.padding=padding


    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        #Select the sentence and label at the specified index in the data frame
        prompt_text_tokens=[]
        prompt_question_tokens=[]
        summary_tokens=[]
        #["text","prompt_question","prompt_text"]

        if(self.split==True):
            prompt_text_tokens=['[CLS]'] + self.tokenizer.tokenize(self.df.loc[index, self.input_cols[2]])
            prompt_question_tokens=['[CLS]'] + self.tokenizer.tokenize(self.df.loc[index, self.input_cols[1]])
            summary_tokens=['[CLS]'] + self.tokenizer.tokenize(self.df.loc[index, self.input_cols[0]])
            if len(prompt_text_tokens) < self.maxlen:
                if self.padding:
                    prompt_text_tokens = prompt_text_tokens + ['[PAD]' for _ in range(self.maxlen - len(prompt_text_tokens))]+ ['[SEP]']
                else: prompt_text_tokens = prompt_text_tokens + ['[SEP]']
            else:
                prompt_text_tokens = prompt_text_tokens[:self.maxlen-1] + ['[SEP]']
            if len(prompt_question_tokens) < self.maxlen:
                if self.padding:
                    prompt_question_tokens = prompt_question_tokens + ['[PAD]' for _ in range(self.maxlen - len(prompt_question_tokens))]+ ['[SEP]']
                else: prompt_question_tokens = prompt_question_tokens + ['[SEP]']
            else:
                prompt_question_tokens = prompt_question_tokens[:self.maxlen-1] + ['[SEP]']
            if len(summary_tokens) < self.maxlen:
                if self.padding:
                    summary_tokens = summary_tokens + ['[PAD]' for _ in range(self.maxlen - len(summary_tokens))]+ ['[SEP]']
                else: summary_tokens = summary_tokens + ['[SEP]']
            else:
                summary_tokens = summary_tokens[:self.maxlen-1] + ['[SEP]']

            #Obtain the indices of the tokens in the BERT Vocabulary
            prompt_text_input_ids = self.tokenizer.convert_tokens_to_ids(prompt_text_tokens)
            prompt_question_tokens_input_ids = self.tokenizer.convert_tokens_to_ids(prompt_question_tokens)
            summary_tokens_input_ids = self.tokenizer.convert_tokens_to_ids(summary_tokens)

            prompt_text_input_ids = torch.tensor(prompt_text_input_ids)
            prompt_question_tokens_input_ids = torch.tensor(prompt_question_tokens_input_ids)
            summary_tokens_input_ids = torch.tensor(summary_tokens_input_ids)
            input_ids=[prompt_text_input_ids, prompt_question_tokens_input_ids, summary_tokens_input_ids]
            input_length=len(input_ids)

        else:
            #Select the sentence and label at the specified index in the data frame
            tokens=[]
            for col in self.input_cols:
              temp_tokens = self.tokenizer.tokenize(self.df.loc[index, col])
              tokens = tokens + temp_tokens + ['[SEP]']
            #Preprocess the text to be suitable for the transformer
            tokens = ['[CLS]'] + tokens
            if len(tokens) < self.maxlen:
                if self.padding:
                  input_length=len(tokens)
                  tokens = tokens + ['[PAD]' for _ in range(self.maxlen - len(tokens))]
            else:
                tokens = tokens[:self.maxlen-1]

            #Obtain the indices of the tokens in the BERT Vocabulary
            input_ids = self.tokenizer.convert_tokens_to_ids(tokens)
            input_ids = torch.tensor(input_ids)
        try:
            target = self.df.loc[index, self.target_cols]
        except Exception as e:
            raise e
        try:
            feature = self.df.loc[index, self.feature_cols]
        except Exception as e:
            raise e
        target = torch.tensor(target, dtype=torch.float32)
        feature = torch.tensor(feature, dtype=torch.float32)
        return input_ids, input_length, target, feature

In [207]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

In [208]:
FEATURE_COLUMNS

['text_word_cnt',
 'text_length',
 'text_stopword_cnt',
 'text_punct_cnt',
 'text_different_word_cnt',
 'text_misspelled_cnt',
 'text_word_ratio',
 'text_length_ratio',
 'text_stopword_ratio',
 'text_punct_ratio',
 'text_different_word_ratio',
 'karp_tfidf_scores',
 '2grams_cnt',
 '3grams_cnt',
 '4grams_cnt']

In [209]:
MAX_LEN=2000
BATCH_SIZE=10
split=False
padding= True
#input cols are the one for LSTM input
# Feature col are the one for dense layer input

input_cols=["text","prompt_question","prompt_text"]
target_cols=["content", "wording"]
train_set = CommonLitDataset(data=train_df, maxlen=MAX_LEN, tokenizer=tokenizer, input_cols=input_cols, feature_cols=FEATURE_COLUMNS, target_cols=target_cols, split=split, padding=padding)
valid_set = CommonLitDataset(data=valid_df, maxlen=MAX_LEN, tokenizer=tokenizer, input_cols=input_cols, feature_cols=FEATURE_COLUMNS, target_cols=target_cols, split=split, padding=padding)
test_set = CommonLitDataset(data=test_df, maxlen=MAX_LEN, tokenizer=tokenizer, input_cols=input_cols, feature_cols=FEATURE_COLUMNS, target_cols=target_cols, split=split, padding=padding)

train_loader = DataLoader(dataset=train_set, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
valid_loader = DataLoader(dataset=valid_set, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
test_loader = DataLoader(dataset=test_set, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)

In [210]:
sample = next(iter(train_loader))

In [211]:
sample


[tensor([[ 101, 1335, 1103,  ...,    0,    0,    0],
         [ 101, 1103, 1934,  ...,    0,    0,    0],
         [ 101, 1409, 1103,  ...,    0,    0,    0],
         ...,
         [ 101, 2677, 3050,  ...,    0,    0,    0],
         [ 101, 1124, 1434,  ...,    0,    0,    0],
         [ 101, 1220, 1156,  ...,    0,    0,    0]]),
 tensor([ 959,  792, 1321, 1273,  734, 1101,  951,  869,  778, 1262]),
 tensor([[ 1.3602, -0.0120],
         [-0.9813, -1.5489],
         [-0.2006, -0.5806],
         [-1.0019,  0.4675],
         [-1.5472, -1.4612],
         [ 2.3328,  1.6292],
         [ 0.7021,  0.6671],
         [ 0.0188, -1.2940],
         [ 0.1805,  1.2948],
         [-1.6385, -0.9120]]),
 tensor([[0.1981, 0.1999, 0.2180, 0.2160, 0.2283, 0.2326, 0.1981, 0.1999, 0.2180,
          0.2160, 0.2283, 0.6807, 0.1736, 0.1529, 0.1220],
         [0.0208, 0.0212, 0.0415, 0.0123, 0.0551, 0.0581, 0.0208, 0.0212, 0.0415,
          0.0123, 0.0551, 0.2723, 0.0207, 0.0000, 0.0000],
         [0.0945, 0.0

In [212]:
train_df.describe()

,content,wording,text_word_cnt,text_length,text_stopword_cnt,text_punct_cnt,text_different_word_cnt,text_misspelled_cnt,text_word_ratio,text_length_ratio,text_stopword_ratio,text_punct_ratio,text_different_word_ratio,karp_tfidf_scores,2grams_cnt,3grams_cnt,4grams_cnt
count,5268.000000,5268.000000,5268.000000,5268.000000,5268.000000,5268.000000,5268.000000,5268.000000,5268.000000,5268.000000,5268.000000,5268.000000,5268.000000,5268.000000,5268.000000,5268.000000,5268.000000
mean,0.008219,0.007689,0.105363,0.103778,0.124174,0.096956,0.150733,0.120480,0.105363,0.103778,0.124174,0.096956,0.150733,0.455593,0.096387,0.066923,0.055980
std,1.050281,1.052079,0.111996,0.107741,0.114502,0.095319,0.123162,0.100499,0.111996,0.107741,0.114502,0.095319,0.123162,0.229925,0.104592,0.097448,0.096170
min,-1.638511,-1.795491,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-0.750933,-0.798164,0.031949,0.032298,0.047826,0.034483,0.063107,0.054054,0.031949,0.032298,0.047826,0.034483,0.063107,0.275213,0.033058,0.008333,0.000000
50%,-0.093814,-0.031987,0.068871,0.068045,0.086957,0.067797,0.116105,0.093023,0.068871,0.068045,0.086957,0.067797,0.116105,0.427350,0.061983,0.027397,0.013514
75%,0.499660,0.547661,0.139691,0.137970,0.160870,0.123457,0.200787,0.153846,0.139691,0.137970,0.160870,0.123457,0.200787,0.619429,0.120690,0.086275,0.073171
max,3.802722,4.310693,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Define model and avaluation functions

In [213]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim1, hidden_dim2, output_dim, n_layers,
                 bidirectional, dropout):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim,
                            hidden_dim1,
                            num_layers=n_layers,
                            bidirectional=bidirectional,
                            batch_first=True)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(hidden_dim1 * 2, hidden_dim2)
        self.fc2 = nn.Linear(hidden_dim2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lengths, feature):
        embedded = self.embedding(text)
        packed_embedded = pack_padded_sequence(embedded, text_lengths.to('cpu'), batch_first=True, enforce_sorted=False).to(device)
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        cat = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        rel = self.relu(cat)
        x=torch.cat((rel, feature), dim=1)
        dense1 = self.fc1(x)
        drop = self.dropout(dense1)
        preds = self.fc2(drop)
        return preds


In [218]:
def evaluate(model, criterion, dataloader, device):
    model.eval()
    mean_acc, mean_loss, count = 0, 0, 0

    with torch.no_grad():
        for input_ids, text_length, target in (dataloader):

            input_ids, text_length, target = input_ids.to(device), text_length.to(device), target.to(device)
            output = model(input_ids, text_length)

            mean_loss += criterion(output, target).item()
            #mean_loss += get_rmse(output, target.type_as(output)).item()
            count += 1

    return mean_loss/count


from tqdm import tqdm, trange

def train(model, criterion, optimizer, train_loader, val_loader, epochs, device):
    best_acc = 0
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for (input_ids, input_length, target, feature) in tqdm(train_loader):
            optimizer.zero_grad()
            feature =feature.to(device)
            input_ids, target = input_ids.to(device), target.to(device)
            input_length= input_length.to(device)
            output = model(input_ids, input_length, feature)

            loss = criterion(output, target.type_as(output))
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        tl = train_loss/len(train_loader)
        val_loss = evaluate(model=model, criterion = criterion, dataloader=val_loader, device=device)
        print(f"Epoch {epoch} - Training Loss: {tl} Validation Loss: {val_loss}")

In [215]:
class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps

    def forward(self, yhat, y):
        loss = torch.sqrt(self.mse(yhat, y) + self.eps)
        return loss


class MCRMSELoss(nn.Module):
    def __init__(self, num_scored=2):
        super().__init__()
        self.rmse = RMSELoss()
        self.num_scored = num_scored

    def forward(self, yhat, y):
        score = 0
        for i in range(self.num_scored):
            score += self.rmse(yhat[:, i], y[:, i]) / self.num_scored

        return score

## Define hyperparameters and train

In [216]:
n_epochs = 250
lr = 0.01
n_folds = 5
lstm_input_size = 1
hidden_state_size = 30
batch_size = 30
num_sequence_layers = 2
output_dim = 11
num_time_steps = 4000
rnn_type = 'LSTM'

In [219]:
LR = 1e-3
device = "cuda" if torch.cuda.is_available() else "cpu"
model = LSTM(vocab_size=30522, embedding_dim=20, bidirectional=True,dropout=0.5,hidden_dim1=100, hidden_dim2=50,n_layers=2,output_dim=2)
model.to(device)
OPTIMIZER = optim.Adam(params=model.parameters(), lr=LR)
EPOCHS = 5
CRITERION = MCRMSELoss()
train(model=model,
      criterion=CRITERION,
      optimizer=OPTIMIZER,
      train_loader=train_loader,
      val_loader=valid_loader,
      epochs = EPOCHS,
     device = device)

  0%|          | 0/527 [00:00<?, ?it/s]


RuntimeError: ignored

In [ ]:
sample[0]

In [ ]:
print(model(sample[0].to(device), sample[1].to(device)))
print(sample[2])

## Define model (NON DOVREBBE SERVIRE)

In [ ]:
class Bi_RNN(nn.Module):

    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=11, num_layers=2, rnn_type='LSTM'):
        super(Bi_RNN, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers

        #Define the initial linear hidden layer
        self.init_linear = nn.Linear(self.input_dim, self.input_dim)

        # Define the LSTM layer
        self.lstm = eval('nn.' + rnn_type)(self.input_dim, self.hidden_dim, self.num_layers, batch_first=True, bidirectional=True)

        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim * 2, output_dim)

    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def forward(self, input):
        #Forward pass through initial hidden layer
        linear_input = self.init_linear(input)

        # Forward pass through LSTM layer
        # shape of lstm_out: [batch_size, input_size ,hidden_dim]
        # shape of self.hidden: (a, b), where a and b both
        # have shape (batch_size, num_layers, hidden_dim).
        lstm_out, self.hidden = self.lstm(linear_input)

        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.linear(lstm_out)
        return y_pred

##Roba in piu'

In [ ]:
print(os.getcwd())
summaries_train_path = "https://raw.githubusercontent.com/Sopralapanca/CommonLitChallenge/main/data/summaries_train.csv"
prompt_train_path = "https://raw.githubusercontent.com/Sopralapanca/CommonLitChallenge/main/data/prompts_train.csv"

summaries_test_path = "https://raw.githubusercontent.com/Sopralapanca/CommonLitChallenge/main/data/summaries_test.csv"
prompt_test_path = "https://raw.githubusercontent.com/Sopralapanca/CommonLitChallenge/main/data/prompts_test.csv"

train_data = pd.read_csv(summaries_train_path, sep=',', index_col=0)
prompt_data = pd.read_csv(prompt_train_path, sep=',', index_col=0)

In [ ]:
training_data = train_data.merge(prompt_data, on='prompt_id')

In [ ]:
training_data

In [ ]:
training_data['prompt_text_length']= training_data['prompt_text'].str.len()
training_data['prompt_question_length']=training_data['prompt_question'].str.len()
training_data['student_summary_length']=training_data['text'].str.len()

In [ ]:
train, test = train_test_split(training_data,  test_size=0.2, shuffle=True)
train, validation = train_test_split(train,  test_size=0.2, shuffle=True)